In [8]:
# source data, can be from web or from local drive

file = r'https://raw.githubusercontent.com/clueple/free_resources/master/sample_data.csv'


In [9]:
""" Import all the necessary libraries  """
import numpy as np
from pandas import read_excel, DataFrame, set_option, read_csv, concat, to_datetime

# use 'set_option' to define the size of the displayed dataframe columns
set_option('display.max_columns',100,'display.width',1000)


In [10]:
# define orginal data source
s_data = read_csv(file)

In [11]:
# preview the last 5 rows of table, s_data 
s_data.tail()

,Date,Open,High,Low,Close,Volume
3067,2022-01-25 00:00:00,24203.0,24358.0,23996.0,24238.0,169458
3068,2022-01-26 00:00:00,24428.0,24477.0,24121.0,24317.0,141282
3069,2022-01-27 00:00:00,24100.0,24100.0,23569.0,23840.0,119772
3070,2022-01-28 00:00:00,23900.0,23909.0,23549.0,23612.0,6968
3071,2022-01-31 00:00:00,23585.0,23958.0,23426.0,23862.0,59520


In [16]:
# generate a dictionary object, 'b_feature', to extrat features from the 's_data' table
b_feature = {
    # daily % change on the 'Close' column 
    'chg': 100*s_data['Close'].pct_change(1).to_numpy(),
    # % change from today's Open to today's Close
    'ichg': 100*(s_data['Close'].sub(s_data['Open'].shift(1)))/(s_data['Open'].shift(1)).to_numpy()
}

In [17]:
# convert dict, 'b_feature' to a dataframe, 'b_feat'
b_feat = DataFrame.from_dict(b_feature)

In [18]:
# preview the table, 'b_feat' by showing the last 5 rows
b_feat.tail()

,chg,ichg
3067,-1.715259,-1.790924
3068,0.325934,0.471016
3069,-1.961591,-2.407074
3070,-0.956376,-2.024896
3071,1.058784,-0.158996


In [19]:
# Let's say I wanna check if the 'chg' column is done right.  Then I slice the 'Close' and 'Date' column from the original table, 's_data'
# I call this sliced table, 'sliced'
sliced = s_data[['Date', 'Close']]


In [20]:
# preview the table, 'sliced' by showing the last 5 rows
sliced.tail()

,Date,Close
3067,2022-01-25 00:00:00,24238.0
3068,2022-01-26 00:00:00,24317.0
3069,2022-01-27 00:00:00,23840.0
3070,2022-01-28 00:00:00,23612.0
3071,2022-01-31 00:00:00,23862.0


In [21]:
# now, let me create a table, 'combined', to include the columns, 'chg', 'ichg', 'Date', and 'Close'
# since the 2 tables, 'b_feat' and 'sliced', have the same index (e.g. 0 to 3071) and the same length (both have 3072 rows), we can simply combine the two
combined = concat([b_feat, sliced], axis=1)

In [22]:
# preview the combined table by showing the last 5 rows
combined.tail()

,chg,ichg,Date,Close
3067,-1.715259,-1.790924,2022-01-25 00:00:00,24238.0
3068,0.325934,0.471016,2022-01-26 00:00:00,24317.0
3069,-1.961591,-2.407074,2022-01-27 00:00:00,23840.0
3070,-0.956376,-2.024896,2022-01-28 00:00:00,23612.0
3071,1.058784,-0.158996,2022-01-31 00:00:00,23862.0


In [25]:
# We may need to combine different tables repeatedly, therefore, we better create a function for this task
# Here I create a function call 'join_multiple_df' for it

def join_multiple_df(*args):  #insert table names separated by commas
    combined = concat([*args], axis=1)
    return combined


In [26]:
# Now, let's try the 'join_multiple_df' function to join the 'b_feat' and 'sliced' tables
# I store the new combined table in a table called, 'n_combined'

n_combined = join_multiple_df(b_feat, sliced)

In [27]:
# Preview the last 5 rows of the table, 'n_combined'

n_combined.tail()

,chg,ichg,Date,Close
3067,-1.715259,-1.790924,2022-01-25 00:00:00,24238.0
3068,0.325934,0.471016,2022-01-26 00:00:00,24317.0
3069,-1.961591,-2.407074,2022-01-27 00:00:00,23840.0
3070,-0.956376,-2.024896,2022-01-28 00:00:00,23612.0
3071,1.058784,-0.158996,2022-01-31 00:00:00,23862.0


In [29]:
# The column 'Date' in the 'n_combined' table is too long.  I'd like to chnage its format to 'yy-mm-dd' (as a datetime object)

n_combined['Date'] = to_datetime(n_combined['Date'], format='%Y-%m-%d')

In [30]:
# let's preview the last 5 rows of the table, 'n_combined'
n_combined.tail()

,chg,ichg,Date,Close
3067,-1.715259,-1.790924,2022-01-25,24238.0
3068,0.325934,0.471016,2022-01-26,24317.0
3069,-1.961591,-2.407074,2022-01-27,23840.0
3070,-0.956376,-2.024896,2022-01-28,23612.0
3071,1.058784,-0.158996,2022-01-31,23862.0
